In [ ]:
import os
import pandas as pd

directory = r"C:\Users\15801\OneDrive\桌面\NU\2025winter\data mining\project\instacart-market-basket-analysis"

all_files = os.listdir(directory)

csv_files = [f for f in all_files if f.endswith(".csv")]
print(csv_files)
dataframes = {}

for file in csv_files:
    if file not in 'user_order_amount_matrix.csv':
        file_path = os.path.join(directory, file, file)
        df = pd.read_csv(file_path)
        dataframes[file] = df
        print(f"read {file}, shape: {df.shape}")

first_file = csv_files[0] if csv_files else None
if first_file:
    print(f"\n sample DataFrame ({first_file}):")
    display(dataframes[first_file])

['aisles.csv', 'departments.csv', 'orders.csv', 'order_products__prior.csv', 'order_products__train.csv', 'products.csv', 'sample_submission.csv', 'user_order_amount_matrix.csv']
read aisles.csv, shape: (134, 2)
read departments.csv, shape: (21, 2)
read orders.csv, shape: (3421083, 7)
read order_products__prior.csv, shape: (32434489, 4)
read order_products__train.csv, shape: (1384617, 4)
read products.csv, shape: (49688, 4)
read sample_submission.csv, shape: (75000, 2)

 sample DataFrame (aisles.csv):


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
...,...,...
129,130,hot cereal pancake mixes
130,131,dry pasta
131,132,beauty
132,133,muscles joints pain relief


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# only using order_data.csv
order_df = dataframes["orders.csv"]
aisles_df = dataframes["aisles.csv"]
department_df = dataframes["departments.csv"]
train_df = dataframes["order_products__prior.csv"]
train_df = order_df.merge(train_df, on="order_id", how="inner")

train_df = train_df.drop_duplicates(subset=['user_id', 'order_id'], keep="first")

In [ ]:
train_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_amount_scaled,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,-0.366918,196,1,1
1,1187899,1,train,11,4,8,14.0,-0.366918,25133,2,1
2,1187899,1,train,11,4,8,14.0,-0.366918,38928,3,1
3,1187899,1,train,11,4,8,14.0,-0.366918,26405,4,1
4,1187899,1,train,11,4,8,14.0,-0.366918,39657,5,1


In [ ]:
order_df = dataframes["orders.csv"]
aisles_df = dataframes["aisles.csv"]
department_df = dataframes["departments.csv"]
train_df = dataframes["order_products__prior.csv"]
train_df = order_df.merge(train_df, on="order_id", how="inner")
print(train_df.shape)
train_df = train_df.drop_duplicates(subset=['user_id', 'order_id', 'product_id'], keep="first")
print(train_df.shape)
train_df = train_df.dropna()
print(train_df.shape)
product_sales = train_df.groupby("product_id")["order_id"].count().reset_index()
product_sales.rename(columns={"order_id": "total_sales"}, inplace=True)
product_sales = product_sales.sort_values(by="total_sales", ascending=False)

popular_products = product_sales[product_sales["total_sales"] >= 500]["product_id"]

user_order_counts = train_df.groupby("user_id")["order_id"].nunique().reset_index()
user_order_counts.rename(columns={"order_id": "total_orders"}, inplace=True)
active_users = user_order_counts[user_order_counts["total_orders"] >= 15]["user_id"]

train_df = train_df[(train_df["product_id"].isin(popular_products)) & (train_df["user_id"].isin(active_users))]

(32434489, 10)
(32434489, 10)
(30356421, 10)


In [ ]:
train_df.head()

     order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
59    2168274        2    prior             1          2                 11   
60    2168274        2    prior             1          2                 11   
61    2168274        2    prior             1          2                 11   
62    2168274        2    prior             1          2                 11   
63    2168274        2    prior             1          2                 11   
..        ...      ...      ...           ...        ...                ...   
249    839880        2    prior            14          3                 10   
250    839880        2    prior            14          3                 10   
251    839880        2    prior            14          3                 10   
252    839880        2    prior            14          3                 10   
253    839880        2    prior            14          3                 10   

     days_since_prior_order  order_amount_scaled  p

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26], dtype=int64)

In [ ]:
print(train_df.shape)
print(len(train_df["user_id"].unique()))
print(len(train_df["product_id"].unique()))

(19418203, 10)
65290
7907


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def df_process(df):
  scaler = StandardScaler()
  df[['order_hour_of_day', 'order_dow', 'days_since_prior_order', "order_number"]] = scaler.fit_transform(
      df[['order_hour_of_day', 'order_dow', 'days_since_prior_order', "order_number"]]
  )
  print("Finished order_hour_scaler")

  """
  df['order_amount_scaled'] = 0
  scalers={}
  for product in df["product_id"].unique():
    scaler = StandardScaler()
    product_mask = df['product_id'] == product
    df.loc[product_mask, 'order_amount_scaled'] = scaler.fit_transform(df.loc[product_mask, ['order_number']])
    scalers[product] = scaler
  print("Finished product count scaler")
  """

  minMaxScaler = MinMaxScaler(feature_range=(0, 1))
  df[['days_since_prior_order_scaled']] = minMaxScaler.fit_transform(
      df[['days_since_prior_order']]
  )
  print("days_since_prior_order_scaled finished")

  df[['days_since_prior_order_scaled']] = 1 - df[['days_since_prior_order_scaled']]
  df['weighted_order_amount'] = df['order_number'] * df['days_since_prior_order_scaled']
  return df

train_df = df_process(train_df)

Finished order_hour_scaler
days_since_prior_order_scaled finished


In [ ]:
np.random.seed(42)
users_sample = np.random.choice(user_order_amount_matrix.index.unique(), 1000, replace=False)

In [ ]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")



PyTorch Version: 2.6.0+cu118
CUDA Available: True
Number of GPUs: 1
GPU Name: NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
train_df = train_df[['user_id', 'product_id', 'weighted_order_amount']].dropna()

# Encode user_id and product_id to unique indices
user_mapping = {user: idx for idx, user in enumerate(train_df["user_id"].unique())}
product_mapping = {product: idx for idx, product in enumerate(train_df["product_id"].unique())}

train_df["user_id"] = train_df["user_id"].map(user_mapping)
train_df["product_id"] = train_df["product_id"].map(product_mapping)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    train_df[["user_id", "product_id"]].values, train_df["weighted_order_amount"].values,
    test_size=0.2, random_state=42
)

# Convert to tensors
X_train_user = torch.tensor(X_train[:, 0], dtype=torch.long)
X_train_product = torch.tensor(X_train[:, 1], dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.float32)

X_test_user = torch.tensor(X_test[:, 0], dtype=torch.long)
X_test_product = torch.tensor(X_test[:, 1], dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create Dataset Class
class OrderDataset(Dataset):
    def __init__(self, users, products, ratings):
        self.users = users
        self.products = products
        self.ratings = ratings

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.products[idx], self.ratings[idx]

# Create DataLoaders
train_dataset = OrderDataset(X_train_user, X_train_product, y_train)
test_dataset = OrderDataset(X_test_user, X_test_product, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define Neural Collaborative Filtering Model
class NCF(nn.Module):
    def __init__(self, num_users, num_products, embedding_dim=64):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.product_embedding = nn.Embedding(num_products, embedding_dim)

        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

        self.relu = nn.ReLU()

    def forward(self, user, product):
        user_emb = self.user_embedding(user)
        product_emb = self.product_embedding(product)
        x = torch.cat([user_emb, product_emb], dim=-1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x.squeeze()

# Initialize Model
num_users = len(user_mapping)
num_products = len(product_mapping)
model = NCF(num_users, num_products).to(device)

# Define Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for users, products, ratings in train_loader:
        users = users.to(device)
        products = products.to(device)
        ratings = ratings.to(device)

        optimizer.zero_grad()
        predictions = model(users, products)
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

# Evaluation
model.eval()
predictions_list = []
actual_list = []

with torch.no_grad():
    for users, products, ratings in test_loader:
        users, products, ratings = users.to(device), products.to(device), ratings.to(device)
        predictions = model(users, products)
        predictions_list.extend(predictions.cpu().numpy())
        actual_list.extend(ratings.cpu().numpy())

rmse = np.sqrt(mean_squared_error(actual_list, predictions_list))
print(f"Test RMSE: {rmse:.4f}")

cuda
Epoch 1, Loss: 0.4342
Epoch 2, Loss: 0.4115
Epoch 3, Loss: 0.4028
Epoch 4, Loss: 0.3948
Epoch 5, Loss: 0.3879
Test RMSE: 0.6336


In [ ]:
rmse = np.sqrt(mean_squared_error(actual_list, predictions_list))
print(f"Test RMSE: {rmse:.4f}")

Test RMSE: 0.6336


In [ ]:
torch.save(model, "ncf_model.pth")

In [ ]:
from sklearn.metrics import ndcg_score

users_sample = np.random.choice(user_order_amount_matrix.index.unique(), 1000, replace=False)

def evaluate_metrics(user_id, K=5):
    user_id = int(user_id)

    if user_id not in user_mapping.values():
        return None, None

    true_orders = np.zeros(num_products)
    user_purchases = train_df[train_df["user_id"] == user_id]["product_id"].values
    for product in user_purchases:
        true_orders[product] = 1

    recommendations = model(torch.tensor([user_id]*K).to(device), torch.tensor(range(K)).to(device))
    recommended_indices = np.argsort(recommendations.cpu().detach().numpy())[::-1][:K]  # 取Top-K推荐

    hr_at_k = int(any(true_orders[i] for i in recommended_indices))

    ideal_dcg = sum([1.0 / np.log2(i + 2) for i in range(min(K, int(true_orders.sum())))])
    dcg = sum([1.0 / np.log2(i + 2) if true_orders[i] > 0 else 0 for i in recommended_indices])
    ndcg_at_k = dcg / ideal_dcg if ideal_dcg > 0 else 0

    return hr_at_k, ndcg_at_k

K_values = list(range(1, 21))
hr_scores = []
ndcg_scores = []

for K in K_values:
    hr_list = []
    ndcg_list = []

    for user in users_sample:
        hr, ndcg = evaluate_metrics(user, K)
        if hr is not None:
            hr_list.append(hr)
        if ndcg is not None:
            ndcg_list.append(ndcg)

    hr_scores.append(np.mean(hr_list))
    ndcg_scores.append(np.mean(ndcg_list))

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(K_values, hr_scores, marker='o', linestyle='-', label="HR@K")
plt.xlabel("K")
plt.ylabel("Hit Rate (HR@K)")
plt.title("HR@K Curve")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(K_values, ndcg_scores, marker='s', linestyle='-', label="NDCG@K")
plt.xlabel("K")
plt.ylabel("NDCG Score (NDCG@K)")
plt.title("NDCG@K Curve")
plt.legend()

plt.tight_layout()
plt.show()
